In [2]:
!pip install langchain-google-genai crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 41.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.2/339.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6

In [24]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

os.environ["SERPER_API_KEY"] = '' # use a serper API

# Initialize the generative model
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose=True,
                             temperature=0.5,
                             google_api_key='' # GEMINI API or OPENAI model
)

search_tool = SerperDevTool(search_query='job description template')


In [25]:
import json  # Import the JSON module to parse JSON strings
from langchain_core.agents import AgentFinish

agent_finishes  = []

import json
from typing import Union, List, Tuple, Dict
from langchain.schema import AgentFinish

call_number = 0

def print_agent_output(agent_output: Union[str, List[Tuple[Dict, str]], AgentFinish], agent_name: str = 'Generic call'):
    global call_number  # Declare call_number as a global variable
    call_number += 1
    with open("crew_callback_logs.txt", "a") as log_file:
        # Try to parse the output if it is a JSON string
        if isinstance(agent_output, str):
            try:
                agent_output = json.loads(agent_output)  # Attempt to parse the JSON string
            except json.JSONDecodeError:
                pass  # If there's an error, leave agent_output as is

        # Check if the output is a list of tuples as in the first case
        if isinstance(agent_output, list) and all(isinstance(item, tuple) for item in agent_output):
            print(f"-{call_number}----Dict------------------------------------------", file=log_file)
            for action, description in agent_output:
                # Print attributes based on assumed structure
                print(f"Agent Name: {agent_name}", file=log_file)
                print(f"Tool used: {getattr(action, 'tool', 'Unknown')}", file=log_file)
                print(f"Tool input: {getattr(action, 'tool_input', 'Unknown')}", file=log_file)
                print(f"Action log: {getattr(action, 'log', 'Unknown')}", file=log_file)
                print(f"Description: {description}", file=log_file)
                print("--------------------------------------------------", file=log_file)

        # Check if the output is a dictionary as in the second case
        elif isinstance(agent_output, AgentFinish):
            print(f"-{call_number}----AgentFinish---------------------------------------", file=log_file)
            print(f"Agent Name: {agent_name}", file=log_file)
            agent_finishes.append(agent_output)
            # Extracting 'output' and 'log' from the nested 'return_values' if they exist
            output = agent_output.return_values
            # log = agent_output.get('log', 'No log available')
            print(f"AgentFinish Output: {output['output']}", file=log_file)
            # print(f"Log: {log}", file=log_file)
            # print(f"AgentFinish: {agent_output}", file=log_file)
            print("--------------------------------------------------", file=log_file)

        # Handle unexpected formats
        else:
            # If the format is unknown, print out the input directly
            print(f"-{call_number}-Unknown format of agent_output:", file=log_file)
            print(type(agent_output), file=log_file)
            print(agent_output, file=log_file)


In [26]:
class Agents():
    def JD_agent(self):
        return Agent(
            role='Job Description Writer Agent',
            goal="""Generate job description based on requirements provided""",
            backstory="""You are a master at synthesizing a variety of Job descriptiond
            that will address the requirements mentioned""",
            llm=llm,
            verbose=True,
            allow_delegation=False,
            max_iter=5,
            memory=True,
            step_callback=lambda x: print_agent_output(x,"Job Description Writer Agent"),
        )

In [12]:
requirements = 'C++ / C# Developer with 3+ years of experience or a 2024 passout fresher with Azure, teraform skills'

In [18]:
class Tasks():

    def draft_JD(self, requirements,jDAgent):
        return Task(
        description=f"""Draft a comprehensive job description based on the gathered requirements. \
            Use the provided requirements to outline the responsibilities, qualifications, and \
            any other relevant details for the position. \

            REQUIREMENTS:\n\n {requirements} \n\n""",
        expected_output="""A well-crafted job description that accurately represents the position \
            and its requirements""",
        context=[],
        agent=jDAgent,
        output_file="draft_job_description.txt",
    )



In [21]:
agents = Agents()
tasks = Tasks()

JDAgent = agents.JD_agent()
task = tasks.draft_JD(requirements,JDAgent)

In [22]:
from crewai import Crew, Process

crew = Crew(
    agents=[JDAgent],
    tasks=[task],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    step_callback=lambda x: print_agent_output(x,"MasterCrew Agent")
)

In [23]:
# Kick off the crew's work
results = crew.kickoff()

 [DEBUG]: == Working Agent: Job Description Writer Agent
 [INFO]: == Starting Task: Draft a comprehensive job description based on the gathered requirements.             Use the provided requirements to outline the responsibilities, qualifications, and             any other relevant details for the position. 
            REQUIREMENTS:

 C++ / C# Developer with 3+ years of experience or a 2024 passout fresher with Azure, teraform skills 




> Entering new CrewAgentExecutor chain...
Final Answer: **Job Title:** C++ / C# Developer

**Job Summary:**

We are seeking a highly skilled C++ / C# Developer with 3+ years of experience or a 2024 passout fresher with Azure and Terraform skills. The ideal candidate will be responsible for developing, maintaining, and enhancing our software applications.

**Responsibilities:**

* Design, develop, and implement software solutions using C++ and C#
* Work closely with other developers and engineers to create high-quality, scalable, and maintainable cod